#Notebook para manipulação dos dados finais

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
import os

BRONZE_PATH = os.path.join("..", "table_destinations", "info_transportes.parquet")
SILVER_PATH = os.path.join("..", "table_destinations", "info_transportes_silver.parquet")
GOLD_PATH = os.path.join("..", "table_destinations", "info_corridas_do_dia.parquet")

spark = SparkSession.builder.appName("assignment1").getOrCreate()
print("Sessão spark iniciada com sucesso")

Sessão spark iniciada com sucesso


In [3]:
df_bronze = spark.read.parquet(BRONZE_PATH)

df_bronze.printSchema()
contagem = df_bronze.count()
print(f"Número de registro bronze: {contagem}")
df_bronze.show()

root
 |-- DATA_INICIO: string (nullable = true)
 |-- DATA_FIM: string (nullable = true)
 |-- CATEGORIA: string (nullable = true)
 |-- LOCAL_INICIO: string (nullable = true)
 |-- LOCAL_FIM: string (nullable = true)
 |-- DISTANCIA: integer (nullable = true)
 |-- PROPOSITO: string (nullable = true)

Número de registro bronze: 1153
+----------------+----------------+---------+-----------------+---------------+---------+-----------------+
|     DATA_INICIO|        DATA_FIM|CATEGORIA|     LOCAL_INICIO|      LOCAL_FIM|DISTANCIA|        PROPOSITO|
+----------------+----------------+---------+-----------------+---------------+---------+-----------------+
|01-01-2016 21:11|01-01-2016 21:17|  Negocio|      Fort Pierce|    Fort Pierce|       51|      Alimentação|
|01-02-2016 01:25|01-02-2016 01:37|  Negocio|      Fort Pierce|    Fort Pierce|        5|             NULL|
|01-02-2016 20:25|01-02-2016 20:38|  Negocio|      Fort Pierce|    Fort Pierce|       48|         Entregas|
|01-05-2016 17:31|01-0

In [ ]:
df_silver = spark.read.parquet(SILVER_PATH)

df_silver.printSchema()
contagem = df_silver.count()
print(f"Número de registro silver: {contagem}")
df_silver.show()

null_count = df_silver.select([f.count(f.when(f.col(c).isNull(), c)).alias(c) for c in df_silver.columns])
null_count.show()

root
 |-- DATA_INICIO: string (nullable = true)
 |-- DATA_FIM: string (nullable = true)
 |-- CATEGORIA: string (nullable = true)
 |-- LOCAL_INICIO: string (nullable = true)
 |-- LOCAL_FIM: string (nullable = true)
 |-- DISTANCIA: long (nullable = true)
 |-- PROPOSITO: string (nullable = true)
 |-- DT_REFE: string (nullable = true)

Número de registro silver: 1153
+----------------+----------------+---------+-----------------+---------------+---------+-----------------+----------+
|     DATA_INICIO|        DATA_FIM|CATEGORIA|     LOCAL_INICIO|      LOCAL_FIM|DISTANCIA|        PROPOSITO|   DT_REFE|
+----------------+----------------+---------+-----------------+---------------+---------+-----------------+----------+
|01-01-2016 21:11|01-01-2016 21:17|  Negocio|      Fort Pierce|    Fort Pierce|       51|      Alimentacao|2016-01-01|
|01-02-2016 01:25|01-02-2016 01:37|  Negocio|      Fort Pierce|    Fort Pierce|        5|       Nao-consta|2016-01-02|
|01-02-2016 20:25|01-02-2016 20:38|  Ne

In [5]:
info_corridas_do_dia = spark.read.parquet(GOLD_PATH)

info_corridas_do_dia.printSchema()
contagem = info_corridas_do_dia.count()
print(f"Número de registro gold: {contagem}")
info_corridas_do_dia.show()
# info_corridas_do_dia.filter(f.col("DT_REFE") == '2016-01-06').show()
# info_corridas_do_dia.filter(f.col("VL_MAX_DIST") > '40.0').show()
# info_corridas_do_dia.filter(f.col("QT_CORR") > 10).show()

root
 |-- DT_REFE: string (nullable = true)
 |-- QT_CORR: long (nullable = true)
 |-- QT_CORR_NEG: long (nullable = true)
 |-- QT_CORR_PESS: long (nullable = true)
 |-- VL_MAX_DIST: long (nullable = true)
 |-- VL_MIN_DIST: long (nullable = true)
 |-- VL_AVG_DIST: double (nullable = true)
 |-- QT_CORR_REUNI: long (nullable = true)
 |-- QT_CORR_NAO_REUNI: long (nullable = true)

Número de registro gold: 287
+----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
|   DT_REFE|QT_CORR|QT_CORR_NEG|QT_CORR_PESS|VL_MAX_DIST|VL_MIN_DIST|VL_AVG_DIST|QT_CORR_REUNI|QT_CORR_NAO_REUNI|
+----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
|2016-08-17|      8|          8|           0|        121|         14|       60.4|            0|                8|
|2016-04-22|      4|          4|           0|        225|         10|      118.8|            4|                0|
|2016-08-08|      5| 